In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain_openai import OpenAI

llm = OpenAI(temperature=0.6)
name = llm.invoke("I want to open a restaurant for Italian food. Suggest a fancy name for this.")
print(name)



"La Dolce Vita Trattoria"


In [3]:
from langchain.prompts import PromptTemplate

prompt_restaurant_name = PromptTemplate(
    input_variables=['cuisine'],
    template="I want to open a restaurant for {cuisine} food. Suggest a fancy name for this."
)

prompt_restaurant_name.format(cuisine='French')

'I want to open a restaurant for French food. Suggest a fancy name for this.'

In [4]:
from langchain.chains import LLMChain

name_chain = LLMChain(llm=llm, prompt=prompt_restaurant_name)

In [5]:
prompt_menu_items = PromptTemplate(
    input_variables=['restaurant_name'],
    template="Suggest some menu items for {restaurant_name}. Return it as a comma separated list."
)
menu_items_chain = LLMChain(llm=llm, prompt=prompt_menu_items)

In [6]:
from langchain.chains import SimpleSequentialChain

sim_seq_chain = SimpleSequentialChain(chains=[name_chain, menu_items_chain])
response = sim_seq_chain.invoke("French")
print(response)

{'input': 'French', 'output': "\n\nLa Belle Cuisine:\n1. Coq au vin\n2. Beef bourguignon\n3. Ratatouille\n4. Escargots\n5. Bouillabaisse\n6. Croque monsieur\n7. Quiche Lorraine\n8. Steak frites\n9. Salade niçoise\n10. Crème brûlée\n\nLe Petit Bistro:\n1. Boeuf bourguignon sandwich\n2. Croque madame\n3. Salade lyonnaise\n4. Moules frites\n5. Quiche du jour\n6. Cassoulet\n7. Poulet rôti\n8. Tarte tatin\n9. Soupe à l'oignon gratinée\n10. Profiteroles au chocolat"}


In [7]:
from langchain.chains import SequentialChain

name_chain = LLMChain(llm=llm, prompt=prompt_restaurant_name, output_key="restaurant_name")
menu_items_chain = LLMChain(llm=llm, prompt=prompt_menu_items, output_key="menu_items")
seq_chain = SequentialChain(
    chains=[name_chain, menu_items_chain],
    input_variables=['cuisine'],
    output_variables=['restaurant_name', 'menu_items']
)

In [8]:
seq_chain.invoke({'cuisine': 'Italian'})

{'cuisine': 'Italian',
 'restaurant_name': '\n\n"Presto Italiano"',
 'menu_items': '\n\n1. Margherita pizza\n2. Fettuccine Alfredo\n3. Chicken Parmigiana\n4. Lasagna\n5. Spaghetti Bolognese\n6. Caprese salad\n7. Cannelloni\n8. Garlic bread\n9. Tiramisu\n10. Gelato\n11. Bruschetta\n12. Ravioli\n13. Penne Arrabiata\n14. Calamari Fritti\n15. Eggplant Parmigiana'}

In [14]:
from langchain.agents import AgentType, initialize_agent, load_tools

tools = load_tools(['wikipedia', 'llm-math'], llm=llm)
agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)
agent.run("When was Elon Musk born? What is his age now in 2024?")



> Entering new AgentExecutor chain...
 I should use wikipedia to find information about Elon Musk's birthdate and his current age.
Action: wikipedia
Action Input: Elon Musk
Observation: Page: Elon Musk
Summary: Elon Reeve Musk (; EE-lon; born June 28, 1971) is a businessman and investor. He is the founder, chairman, CEO, and CTO of SpaceX; angel investor, CEO, product architect, and former chairman of Tesla, Inc.; owner, executive chairman, and CTO of X Corp.; founder of the Boring Company and xAI; co-founder of Neuralink and OpenAI; and president of the Musk Foundation. He is one of the wealthiest people in the world, with an estimated net worth of US$213 billion as of February 2024, according to the Bloomberg Billionaires Index, and $210 billion according to Forbes, primarily from his ownership stakes in Tesla and SpaceX.A member of the wealthy South African Musk family, Elon was born in Pretoria and briefly attended the University of Pretoria before immigrating to Canada at age 18

"Elon Musk's age in 2024 will be 53 years old."

In [16]:
from secret_key import serp_api_key
import os

os.environ['SERPAPI_API_KEY'] = serp_api_key

In [19]:
llm = OpenAI(temperature=0)
tools = load_tools(['serpapi', 'llm-math'], llm=llm)
agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)
agent.run("What was the GDP of USA in 2022 plus 5?")



> Entering new AgentExecutor chain...
 I should use a calculator to add 5 to the GDP of USA in 2022
Action: Calculator
Action Input: GDP of USA in 2022 + 5

ValueError: LLMMathChain._evaluate("
GDP_2022 + 5
") raised error: 'GDP_2022'. Please try again with a valid numerical expression

In [23]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory()
name_chain = LLMChain(llm=llm, prompt=prompt_restaurant_name, memory=memory)
name = name_chain.invoke("Mexican")
print(name)

{'cuisine': 'Mexican', 'history': '', 'text': '\n\n"El Sabor de México" (The Flavor of Mexico)'}


In [24]:
name = name_chain.invoke("French")
print(name)

{'cuisine': 'French', 'history': 'Human: Mexican\nAI: \n\n"El Sabor de México" (The Flavor of Mexico)', 'text': '\n\n"La Belle Cuisine"'}


In [25]:
name_chain.memory

ConversationBufferMemory(chat_memory=ChatMessageHistory(messages=[HumanMessage(content='Mexican'), AIMessage(content='\n\n"El Sabor de México" (The Flavor of Mexico)'), HumanMessage(content='French'), AIMessage(content='\n\n"La Belle Cuisine"')]))

In [26]:
print(name_chain.memory.buffer)

Human: Mexican
AI: 

"El Sabor de México" (The Flavor of Mexico)
Human: French
AI: 

"La Belle Cuisine"


In [27]:
from langchain.chains import ConversationChain

convo = ConversationChain(llm=OpenAI(temperature=0.7))
print(convo.prompt.template)

The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
{history}
Human: {input}
AI:


In [28]:
convo.invoke("Who won the first football world cup?")

{'input': 'Who won the first football world cup?',
 'history': '',
 'response': ' The first football world cup was won by Uruguay in 1930. They defeated Argentina in the final with a score of 4-2. The tournament was held in Uruguay and was the first of its kind, with 13 teams participating. The final match was played in front of a crowd of 93,000 people.'}

In [29]:
convo.invoke("What is 5 + 5?")

{'input': 'What is 5 + 5?',
 'history': 'Human: Who won the first football world cup?\nAI:  The first football world cup was won by Uruguay in 1930. They defeated Argentina in the final with a score of 4-2. The tournament was held in Uruguay and was the first of its kind, with 13 teams participating. The final match was played in front of a crowd of 93,000 people.',
 'response': '  5 + 5 equals 10. This is a basic arithmetic equation that is taught in elementary school. The concept of addition is the process of combining two or more numbers to get a total or sum. In this case, 5 and 5 are added together to get a total of 10. This is a commonly used equation in mathematics and can be applied to various real-life situations.'}

In [30]:
convo.invoke("Who was the captain of the winning team?")

{'input': 'Who was the captain of the winning team?',
 'history': 'Human: Who won the first football world cup?\nAI:  The first football world cup was won by Uruguay in 1930. They defeated Argentina in the final with a score of 4-2. The tournament was held in Uruguay and was the first of its kind, with 13 teams participating. The final match was played in front of a crowd of 93,000 people.\nHuman: What is 5 + 5?\nAI:   5 + 5 equals 10. This is a basic arithmetic equation that is taught in elementary school. The concept of addition is the process of combining two or more numbers to get a total or sum. In this case, 5 and 5 are added together to get a total of 10. This is a commonly used equation in mathematics and can be applied to various real-life situations.',
 'response': ' The captain of the winning team was José Nasazzi, who played for Uruguay. He was a key player in the tournament and also went on to captain the team in the next World Cup in 1934. Nasazzi was known for his strong

In [31]:
print(convo.memory.buffer)

Human: Who won the first football world cup?
AI:  The first football world cup was won by Uruguay in 1930. They defeated Argentina in the final with a score of 4-2. The tournament was held in Uruguay and was the first of its kind, with 13 teams participating. The final match was played in front of a crowd of 93,000 people.
Human: What is 5 + 5?
AI:   5 + 5 equals 10. This is a basic arithmetic equation that is taught in elementary school. The concept of addition is the process of combining two or more numbers to get a total or sum. In this case, 5 and 5 are added together to get a total of 10. This is a commonly used equation in mathematics and can be applied to various real-life situations.
Human: Who was the captain of the winning team?
AI:  The captain of the winning team was José Nasazzi, who played for Uruguay. He was a key player in the tournament and also went on to captain the team in the next World Cup in 1934. Nasazzi was known for his strong leadership skills and excellent d

#### But ConversationBufferMemory will grow endlessly, which will increase the cost of calling API with the entire history!

In [32]:
from langchain.memory import ConversationBufferWindowMemory

short_mem = ConversationBufferWindowMemory(k=1)
short_convo = ConversationChain(llm=OpenAI(temperature=0.7), memory=short_mem)
short_convo.invoke("Who won the first football world cup?")

{'input': 'Who won the first football world cup?',
 'history': '',
 'response': ' The first football world cup was held in 1930 and was won by Uruguay. It was a very exciting event, and Uruguay was the host country. They beat Argentina in the final match with a score of 4-2. Did you know that the football world cup is the most widely viewed and followed sporting event in the world? It is organized by FIFA, the international governing body of football.'}

In [33]:
short_convo.invoke("What is 5 + 5?")

{'input': 'What is 5 + 5?',
 'history': 'Human: Who won the first football world cup?\nAI:  The first football world cup was held in 1930 and was won by Uruguay. It was a very exciting event, and Uruguay was the host country. They beat Argentina in the final match with a score of 4-2. Did you know that the football world cup is the most widely viewed and followed sporting event in the world? It is organized by FIFA, the international governing body of football.',
 'response': ' 5 + 5 equals 10. Did you know that mathematics is the study of numbers, quantities, and shapes? It is a fundamental discipline that is used in many fields such as science, engineering, and finance. '}

In [34]:
short_convo.invoke("Who was the captain of the winning team?")

{'input': 'Who was the captain of the winning team?',
 'history': 'Human: What is 5 + 5?\nAI:  5 + 5 equals 10. Did you know that mathematics is the study of numbers, quantities, and shapes? It is a fundamental discipline that is used in many fields such as science, engineering, and finance. ',
 'response': " I'm sorry, I do not have enough context to answer that question. Could you provide me with more information?"}